In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import datetime
import pickle
import time
import os
import pathlib
import matplotlib.pyplot as plt
import sys
 
# Some stuff to make utils-function work
sys.path.append('../utils')
from pipeline import *
from create_model import *
from utils import *
from unlabeled_utils import *
from evaluate_model import *
%load_ext autoreload
%autoreload 2

# Jupyter-specific
%matplotlib inline

In [ ]:
def plot_and_save(lab_list, show=False):
    lab_array = np.asarray(lab_list, dtype=np.uint8)
    findings = np.bincount(lab_array, minlength=int(conf["num_classes"]))
    print_bar_chart(
        data=[findings],
        conf=conf,
        title="Relevant samples found in unlabeled dataset",
        fname="pseudo_labels_distribution",
        figsize=(16,7),
        show=show
    )
    lab_array=None

In [ ]:
def print_bar_chart(data, conf, title=None, fname=None, figsize=(15,6), show=True):
    """
    Takes in list of data and makes a bar chart of it.
    Dynamically allocates placement for bars.
    """
    import seaborn as sns
    sns.set()

    SMALL_SIZE = 12
    MEDIUM_SIZE = 14
    BIGGER_SIZE = 16

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    x = np.arange(conf["num_classes"])
    width = 0.7      # 1.0 = bars side by side
    width = width/len(data)

    num_bars = len(data)
    if num_bars == 1:
        bar_placement = [0]
    # even number of bars
    elif (num_bars % 2) == 0:
        bar_placement = np.arange(-num_bars/2, num_bars/2+1)    #[-2, -1, 0, 1, 2]
        bar_placement = np.delete(bar_placement, num_bars//2)   #delete 0
        bar_placement = [bar+0.5 if bar<0 else bar-0.5 for bar in bar_placement]
    # odd number of bars
    else:
        bar_placement = np.arange(-np.floor(num_bars/2), np.floor(num_bars/2)+1)

    fig, ax = plt.subplots(figsize=figsize)

    rects = []
    for cnt, (dat, placement) in enumerate(zip(data, bar_placement)):
        rects.append(ax.bar(x+placement*width, dat, width, label='Number of samples in class'))

    ax.set_ylabel('Number of samples')
    if title:
        title_string = title
        ax.set_title(title_string)
    ax.set_xticks(x)
    ax.set_xticklabels(conf["class_names"])
    ax.set_axisbelow(True)
    ax.legend(loc='upper left');

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=25, ha="right",
                 rotation_mode="anchor")
    plt.grid(axis='x')

    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = int(rect.get_height())
            ax.annotate('{}'.format(height),
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),  # 3 points vertical offset
                            textcoords="offset points",
                            ha='center', va='bottom')

    # autolabel(rects1)
    autolabel(rects[-1])

    fig.tight_layout()
    if fname:
        plt.savefig('{}/{}.pdf'.format(conf["log_dir"], fname), format='pdf')
    if show:
        plt.show()
    else:
        plt.close()

In [ ]:
pickle_path = "/home/henriklg/master-thesis/code/hyper-kvasir/experiments/model-size/all-b4/0_student"
pseudo = pickle.load (open (pickle_path+"/unlab_findings.pkl", "rb"))

In [ ]:
conf = {
    "num_classes": 23,
    "log_dir": ".",
    "keep_threshold": 0.9,
    "class_names": ['barretts-short-segment', 'retroflex-stomach', 'ulcerative-colitis-0-1', 'ulcerative-colitis-grade-3', 'esophagitis-b-d', 'dyed-resection-margins', 'hemorrhoids', 'normal-z-line', 'esophagitis-a', 'ulcerative-colitis-1-2', 'barretts', 'bbps-2-3', 'ileum', 'bbps-0-1', 'impacted-stool', 'cecum', 'ulcerative-colitis-grade-2', 'ulcerative-colitis-2-3', 'pylorus', 'retroflex-rectum', 'ulcerative-colitis-grade-1', 'polyps', 'dyed-lifted-polyps'],
}

In [ ]:
plot_and_save(pseudo["lab_list"], show=True)